### Day 1

In [1]:
from itertools import combinations
import numpy as np

In [2]:
with open('day1.txt') as f:
    inputs = [int(x) for x in f.read().splitlines()]

In [3]:
# part 1
[np.prod(x) for x in combinations(inputs, 2) if sum(x) == 2020][0]

910539

In [4]:
# part 2
[np.prod(x) for x in combinations(inputs, 3) if sum(x) == 2020][0]

116724144

### Day 2

In [5]:
with open('day2.txt') as f:
    lines = f.read().splitlines()
    
def clean_txt(text):
    text = text.replace('-', ' ').replace(':', '')
    return text.split(' ')

data = [clean_txt(x) for x in lines]

data = [{
    '_min': int(x[0]),
    '_max': int(x[1]),
    'target':  x[2],
    'example': x[3],
} for x in data]

In [6]:
# part 1
def validate(_min, _max, target, example):
    return _min <= example.count(target) <= _max

sum([validate(**x) for x in data])

564

In [7]:
# part 2
def check_pos(_min, _max, target, example):
    mn = example[_min-1] == target
    mx = example[_max-1] == target
    return mn != mx


sum([check_pos(**x) for x in data])

325

### Day 3

In [8]:
class Toboggan:
    def __init__(self, x_step=3, y_step=1):
        self.x_step = x_step
        self.y_step = y_step
        self.x = 0
        self.y = 0
        with open('day3.txt') as f:
            self.arr = f.read().splitlines()
        
    def move(self):
        self.x = (self.x + self.x_step) % len(self.arr[0])
        self.y += self.y_step

    def check_for_tree(self):
        return int(self.arr[self.y][self.x] == '#')


    
def run_route(x_step, y_step):
    tob = Toboggan(x_step, y_step)
    tree_counter = tob.check_for_tree()

    while tob.y < len(tob.arr) - 1:
        tob.move()
        tree_counter += tob.check_for_tree()

    return tree_counter
    

run_route(3, 1)

230

In [9]:
# part 2
pairs = [
    (1, 1),
    (3, 1),
    (5, 1),
    (7, 1),
    (1, 2),
]

np.prod([run_route(x, y) for x, y in pairs])

9533698720

###  Day 4

In [10]:
with open('day4.txt') as f:
    data = f.read().splitlines()
    
passports_raw = []
item = []
for line in data:
    if line !=  '':
        item += line.strip().split(' ')
    else:
        passports_raw.append(item)
        item = []
passports_raw.append(item) # and final item
        
passports = [{
    itm.split(':')[0]:itm.split(':')[1] for itm in passport
} for passport in passports_raw]

required = {
    'byr',
    'iyr',
    'eyr',
    'hgt',
    'hcl',
    'ecl',
    'pid',
    # 'cid',
}

def check_keys(p):
    return len(required.intersection(set(p.keys()))) == len(required)

sum([check_keys(p) for p in passports])

170

In [11]:
# part 2
import re

def check_rules(p):
    rules = {
        'byr': lambda x: (1920 <= int(x) <= 2002) and (len(x) == 4),
        'iyr': lambda x: (2010 <= int(x) <= 2020) and (len(x) == 4),
        'eyr': lambda x: (2020 <= int(x) <= 2030) and (len(x) == 4),
        'hgt': lambda x: (x[-2:] == 'cm' and 150 <= int(x[:-2]) <=193) 
                         or (x[-2:] == 'in' and 59 <= int(x[:-2]) <= 76),
        'hcl': lambda x: re.search(r'^#(?:[0-9a-fA-F]{6})$', x) is not None,
        'ecl': lambda x: x in ('amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'),
        'pid': lambda x: len(x) == 9 and x.isdigit(),
    }
    return all([rules[r](p[r]) for r in rules])

sum([check_keys(p) and check_rules(p) for p in passports])

103

### Day 5

In [12]:
with open('day5.txt') as f:
    data = f.read().splitlines()

def determine_seat(text):
    """ Convert text to binary int and calc seat id """
    row = int(text[:7].replace('F', '0').replace('B', '1'), 2)
    col = int(text[-3:].replace('L', '0').replace('R', '1'), 2)
    seat_id = 8*row + col
    
    return seat_id

max([determine_seat(x) for x in data])

953

In [13]:
all_seat_ids = range(0, 1024)
seats_taken = [determine_seat(x) for x in data]

[x for x in all_seat_ids if x not in seats_taken and x+1 in seats_taken and x-1 in seats_taken]

[615]

### Day 6

In [14]:
with open('day6.txt') as f:
    lines = f.readlines()

# part 1 - sum of unique questions answered per group
sum([len(set(x.replace('\n', ''))) for x in ''.join(lines).split('\n\n')])

7110

In [15]:
# part 2 - sum of count of questions answered by ALL in group
answers = [x.split('\n') for x in ''.join(lines).split('\n\n')]

for a in range(len(answers)):
    answers[a] = set.intersection(*[set(x) for x in answers[a]])

sum([len(x) for x in answers])

3628

### Day 7

In [16]:
import networkx as nx

In [17]:
with open('day7.txt') as f:
    lines = f.read().splitlines()
    
def prep_line(text):
    items = text.replace('.', '')\
                .replace('bags', '')\
                .replace('bag', '')\
                .replace('  ', ' ')\
                .split('contain')
    
    parent = items[0].strip()
    children = [{
        'colour': x.strip()[2:], 
        'count': int(x.strip()[0])
    } for x in items[1].split(',')]
    
    return parent, children

data = [prep_line(l) for l in lines if 'other' not in l]
data = {d[0]: d[1] for d in data}

connections = []
for col in data.keys():
    for itm in data[col]:
        connections.append((col, itm['colour'], {'count': itm['count']}))

In [18]:
G = nx.DiGraph()
G.add_weighted_edges_from(connections)

In [19]:
# part 1 - how many unique bag types can contain 'shiny gold'
len(set([x[0] for x in nx.edge_dfs(G,'shiny gold', orientation='reverse')]))

142

In [20]:
# part 2 - how many individual bags are required inside your single shiny gold bag?
def find_cost(G, node):
    cost = 0
    
    if G.out_degree(node)==0:
        return 0
    else:
        for x in G.successors(node):
            mult = G[node][x]['weight']['count']
            cost += mult + mult*find_cost(G, x)
        return cost

find_cost(G, 'shiny gold')

10219

### Day 8

In [41]:
class Program:
    def __init__(self):
        self.accumulator = 0
        self.line = 0
        
        with open('day8.txt') as f:
            self.commands = lines = [{
                    'run': False, 
                    'instr': x.split(' ')[0], 
                    'value':  int(x.split(' ')[1])
                } for x in f.read().splitlines()]
        

    def execute_commands(self):
        while not self.commands[self.line]['run']:
            if self.commands[self.line]['instr'] == 'nop':
                self.commands[self.line]['run'] = True
                self.line += 1
                
                
            if self.commands[self.line]['instr'] == 'acc':
                self.accumulator += self.commands[self.line]['value']
                self.commands[self.line]['run'] = True
                self.line += 1
                
            if self.commands[self.line]['instr'] == 'jmp':
                self.commands[self.line]['run'] = True
                self.line += self.commands[self.line]['value']
            
        return self.accumulator


p = Program()
p.execute_commands()

2034

In [61]:
# part 2 - does changing one command make a finite program

class Program:
    def __init__(self, change):
        self.accumulator = 0
        self.line = 0
        
        with open('day8.txt') as f:
            self.commands = lines = [{
                    'run': False, 
                    'instr': x.split(' ')[0], 
                    'value':  int(x.split(' ')[1])
                } for x in f.read().splitlines()]

    def execute_commands(self):
        counter = 0
        check = self.commands[self.line]['run']
        
        while not check and self.line < len(self.commands):
            if counter == self.change_rule_num:
                mapper = {
                    'nop': 'jmp',
                    'jmp': 'nop',
                    'acc': 'acc'
                }
                self.commands[self.line]['instr'] = mapper[self.commands[self.line]['instr']]
            
            if self.commands[self.line]['instr'] == 'nop':
                self.commands[self.line]['run'] = True
                self.line += 1
                
            elif self.commands[self.line]['instr'] == 'acc':
                self.accumulator += self.commands[self.line]['value']
                self.commands[self.line]['run'] = True
                self.line += 1
                
            elif self.commands[self.line]['instr'] == 'jmp':
                self.commands[self.line]['run'] = True
                self.line += self.commands[self.line]['value']
            
            counter += 1
            if self.line < len(self.commands):
                check = self.commands[self.line]['run']
            else:
                break
                
        if self.line == len(self.commands):
            return self.accumulator

for r in range(10000s):
    p = Program(r)

    x = p.execute_commands()
    
    if x:
        print(x, r)
        break

672 52


### Day 9

In [93]:
from itertools import combinations


class Stream:
    def __init__(self, start):
        self.idx = start
        with open('day9.txt') as f:
            self.data = [int(x) for x in f.read().splitlines()]
        self.max = len(self.data)
    
    def check_currently_valid(self):
        check = False

        for a, b in combinations(self.data[self.idx-25:self.idx], 2):
            if a + b == self.data[self.idx]:
                check = True

        return check


s = Stream(25)
while s.idx < s.max:
    if s.check_currently_valid():
        s.idx += 1
    else:
        print(s.data[s.idx])
        break
        
print(s.idx)

27911108
509


In [99]:
target = 27911108
s = Stream(0)

for idx in range(1000):
    count_to_add = 0
    sm = 0
    
    while sm < target:
        sm = sum(s.data[s.idx: s.idx + count_to_add])
        count_to_add += 1
    if sm == target:
        print(min(s.data[s.idx: s.idx + count_to_add]) + max(s.data[s.idx: s.idx + count_to_add]))
        break
    else:
        s.idx += 1

396 18
4023754
